### Overview

In this notebook we introduce the dataset briefly and the base model used.


Relative player ability is modelled as:

$$a_i \sim \mathcal{N}(b \, * \, \text{prior_score}_i, \sigma_a)$$

and coded in Stan as:

```
a = b * prior_score + sigma_a * raw_a;
``` 

**What is the difference between `raw_a` and `a` for a player?**


`raw_a` is like an unpooled estimate of relative player ability based on their data alone. As part of the hierarchical model it represents the per-player adjustment (both mean and std) on top of the shared information contained in the prior and `sigma_a`.


`a` is thus partially-pooled estimate after taking advantage of the shared information we have i.e. by learning both `sigma_a` which tells us how variable player ability is and `b` which tells us how informative the prior score is. 

Both `sigma_a` and `b` are learned across all players and so "share" information.